In [1]:
# !pip install nueramic-mathml

In [1]:
from nueramic_mathml.multi_optimize import gd_optimal
from nueramic_mathml.ml.metrics import binary_classification_report, roc_curve_plot
import torch
import pandas as pd

In [2]:
help(gd_optimal)

Help on function gd_optimal in module nueramic_mathml.multi_optimize:

gd_optimal(function: 'Callable[[torch.Tensor], torch.Tensor]', x0: 'torch.Tensor', epsilon: 'float' = 1e-05, max_iter: 'int' = 500, verbose: 'bool' = False, keep_history: 'bool' = False) -> 'Tuple[torch.Tensor, HistoryGD]'
    Returns a tensor n x 1 with optimal point and history using
    Algorithm with optimal step.
    The idea is to choose a gamma that minimizes the function in the direction :math:`\ f'(x_k)`
    
    
    
    :param function: callable that depends on the first positional argument
    :param x0: Torch tensor which is initial approximation
    :param epsilon: optimization accuracy
    :param max_iter: maximum number of iterations
    :param verbose: flag of printing iteration logs
    :param keep_history: flag of return history
    :return: tuple with point and history.
    
    .. rubric:: Examples
    
    .. code-block:: python3
    
        >>> def func(x): return -torch.exp(- x[0] ** 2 - x[

In [38]:
# Создание данных

# Настройка генератора случайных чисел
torch.random.manual_seed(191)  # для воспроизводимости результатов


x = torch.randint(0, 101, (50, 4)).float() / 50  # псевдо-случайные числа
a = torch.tensor([[1], [-2], [-3], [4.]])  # параметры линейной части
b = 1

y = ((x @ a + b > 0) & 1).flatten().double()  # Создаем y
x = x.double()
y

tensor([1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
        1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
        1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0.],
       dtype=torch.float64)

In [39]:
w0 = torch.zeros(5).double()  # Создаем начальное приближение  cat(a, b)
w0

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [40]:
function_loss = torch.nn.BCELoss()  # Создаем ошибку бинарной кросс-энтропия

In [41]:
# Создадим обертку над функцией ошибки

def minim_f(w):
    y_pred_prob = x @ w[:4].reshape(-1, 1) + w[4]  # x @ ~a + ~b. ~a = w[:4], ~b = w[4] 
    # Применяем сигмоид чтобы посчитать вероятность принадлежности к первому классу
    y_pred_prob = torch.sigmoid(y_pred_prob).flatten()  
    
    return function_loss(y_pred_prob, y)

minim_f(w0)

tensor(0.6931, dtype=torch.float64)

In [46]:
w_opt, hist = gd_optimal(minim_f, w0, keep_history=True, max_iter=10)  # За 10 итераций оценим параметры
w_opt

tensor([ 0.0352, -0.5227, -0.5463,  1.2629, -0.0429], dtype=torch.float64)

In [47]:
y_pred_probs = torch.sigmoid(x @ w_opt[:4].reshape(-1, 1) + w_opt[4])  # Вероятности
y_pred = (y_pred_probs > 0.5) * 1  # Предсказания первого класса

In [48]:
pd.Series(binary_classification_report(y, y_pred, y_pred_probs))  # Отчет о классификации

recall       0.923077
precision    0.960000
accuracy     0.940000
f1           0.941176
auc_roc      0.963141
dtype: float64

In [49]:
roc_curve_plot(y, y_pred_probs)  # ROC - кривая 

In [50]:
print(f'~ a = {w_opt[:4].numpy()}')
print(f'~ b = {w_opt[4].numpy(): 0.3f}')

~ a = [ 0.03517686 -0.52272377 -0.54629736  1.26287008]
~ b = -0.043
